In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Custom Dataset class for loading NPY files
class PointCloudDataset(Dataset):
    def __init__(self, root_dir, n_points=20000):
        self.root_dir = root_dir
        self.n_points = n_points
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            for file_name in os.listdir(class_dir):
                if file_name.endswith('.npy'):
                    file_path = os.path.join(class_dir, file_name)
                    # Check if point cloud has enough points
                    point_cloud = np.load(file_path)
                    if len(point_cloud) >= self.n_points:
                        self.samples.append((
                            file_path,
                            self.class_to_idx[class_name]
                        ))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        point_cloud = np.load(file_path)
        
        # Randomly sample exactly n_points
        indices = np.random.choice(len(point_cloud), self.n_points, replace=False)
        point_cloud = point_cloud[indices]
        
        point_cloud = torch.FloatTensor(point_cloud)
        return point_cloud, label

# T-Net (Input Transform)
class TNet(nn.Module):
    def __init__(self, k=6):  # Changed from k=3 to k=6 to handle all features
        super().__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)

    def forward(self, x):
        batch_size = x.size(0)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        identity = torch.eye(self.k, requires_grad=True).repeat(batch_size, 1, 1)
        if x.is_cuda:
            identity = identity.cuda()
        x = x.view(-1, self.k, self.k) + identity
        return x

# PointNet Feature Extraction
class PointNetFeatures(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_transform = TNet(k=6)  # Changed from k=3 to k=6
        self.feature_transform = TNet(k=64)
        
        self.conv1 = nn.Conv1d(6, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)

    def forward(self, x):
        n_pts = x.size(2)
        
        # Input transformation
        trans = self.input_transform(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        
        # MLP (64,64)
        x = F.relu(self.conv1(x))
        
        # Feature transformation
        trans_feat = self.feature_transform(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans_feat)
        x = x.transpose(2, 1)
        
        # MLP (64,128,1024)
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        
        # Max pooling
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        
        return x

# Function to extract features from the dataset
def extract_features(model, dataloader, device):
    model.eval()
    features = []
    labels = []
    
    with torch.no_grad():
        for points, label in tqdm(dataloader):
            points = points.transpose(2, 1).to(device)
            feat = model(points)
            features.append(feat.cpu().numpy())
            labels.append(label.numpy())
            
    features = np.vstack(features)
    labels = np.concatenate(labels)
    return features, labels

# Main execution

    # Hyperparameters



In [11]:
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create dataset and dataloader
dataset = PointCloudDataset(r'PCDnpy')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# Now 'features' and 'labels' can be used for SVM training


In [12]:

# Initialize model
model = PointNetFeatures().to(device)

# Extract features
features, labels = extract_features(model, dataloader, device)

100%|██████████| 153/153 [18:08<00:00,  7.11s/it]


In [13]:
import numpy as np

# Load the array from the .npy file
file_path = r'features.npy'
np.save(file_path, features)
# Get the shape of the array
file_path = r'labels.npy'
np.save(file_path, labels)



In [14]:
features.shape

(4892, 1024)